In [ ]:
# mou give me 32 thyroid annotations to train a model, which labeled by xiaobai's tool
# I will use python to parse to labelme's syntax.
#from 21/08/21
# 22/01/29 thyroid nodule's modle format valid.

In [ ]:
f1=r"/mnt/d/workspace/03-annotations/1.3.6.1.4.1.52026.66519049.5308.54-v1/1.3.6.1.4.1.52026.66519049.5308.55.1.1.json"

In [ ]:
import json
with open(f1, 'r') as f:
    #data=f.read()
    j1 = json.load(f)
    print(len(j1))
    for i in j1:
        print(i)

In [ ]:
all_imgs_datas = j1['data']
print(len(all_imgs_datas), type(all_imgs_datas))
print(type(all_imgs_datas[0]))
j_data_items = all_imgs_datas[0]
for i in j_data_items:
    print(i)
print(j_data_items['index'])

In [ ]:
uids=[]
for data in all_imgs_datas:
    uids.append(int(data['sopUid'].split('.')[-1]))
uids.sort()
print(uids)

In [ ]:
# get the absent annotations index
j = 0
for i in range(len(uids)):
    if i != uids[j]:
        print(i)
        #j=j-1
    else:
        j =j+1

In [ ]:
j_target_s="""
{
  "shapes": [
    {
      "label": "\u9888\u52a8\u8109:CA",
      "line_color": null,
      "fill_color": null,
      "points": [
        [
          296,
          253
        ],
        [
          295,
          373
        ],
        [
          430,
          363
        ],
        [
          427,
          255
        ]
      ]
    }
  ],
  "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ],
  "imagePath": "frm-0001.png",
  "imageData": null
}
"""

In [ ]:
j_target_o = json.loads(j_target_s)

In [ ]:
print(j_target_o)

In [ ]:
import numpy
one_img = all_imgs_datas[0]
for item in one_img:
    print(item, type(item))
one_img_object = one_img['object']
print(type(one_img_object), len(one_img_object))

for img_data in all_imgs_datas:
    img_objs = img_data['object']
    n = int(img_data['sopUid'].split('.')[-1])
    print(f"img({n}) has {len(img_objs)} object.")
    polygonPoints = img_objs[0]['polygonPoint']
    print(f"polygonPoint length={len(polygonPoints)}")
    filename = f"frm-{(n+1):04d}"
    points_len = int((len(polygonPoints)))
    
    all_points = numpy.zeros((points_len, 2))
    all_points = all_points.tolist()
    print(len(all_points))
    for item in polygonPoints.items():
        #print("point item=", item, "len=", len(item))
        point_idx = int(item[0][1:])
        x0y1 = item[0][0]
        if 'x' == x0y1.lower():
            x0y1 = 0
        else:
            x0y1=1
        #print(f"point idx={point_idx}, coordinate={x0y1}")
        all_points[point_idx][x0y1] = int(item[1])
        #print(f">>>>>point[{point_idx}]=",all_points[int(point_idx)], ">>>>", all_points[point_idx][x0y1])
        #print(f"#all points={all_points[:20][:]}")
        #if (point_idx>10):
        #    break
    
    #print(f"all_points len={len(all_points)}")
    available_pts = []
    for pt in range(points_len):
        #print(f"point [{pt}]={all_points[pt]}")
        if all_points[pt][0] is None or all_points[pt][0]<10:
            #print(f"point[{pt}] is {all_points[pt]} delete it...")
            continue
        else:
            available_pts.append(all_points[pt])
            
    j_target_o['imagePath']=f"{filename}.png"
    j_target_o['shapes'][0]['label']='Thyroid'
    j_target_o['shapes'][0]['points']=available_pts
    
    with open(f"{filename}.json", 'w') as jfp:
        json.dump(j_target_o, jfp)
    
    #break

In [ ]:
!pwd

In [ ]:
# the full function to generate labelme json from xiaobai's json
import json 
import os

j_target_s="""
{
  "shapes": [
    {
      "label": "\u9888\u52a8\u8109:CA",
      "line_color": null,
      "fill_color": null,
      "points": [
        [
          296,
          253
        ],
        [
          295,
          373
        ],
        [
          430,
          363
        ],
        [
          427,
          255
        ]
      ]
    }
  ],
  "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ],
  "imagePath": "frm-0001.png",
  "imageData": null
}
"""


def convert_to_labelme_format(xb_json_file, param):
    """
        deal with one xiaobai's json which have multi-labelme json
    """
    if not os.path.isfile(xb_json_file):
        print(f"ERR: json file:{xb_json_file} not valid")
        return -1
    with open(xb_json_file, 'r') as jfp:
        jdata = json.load(jfp)
    all_imgs_datas = jdata['data']
    series_instance_uid = jdata['series_instance_uid']
    #--01 create home directory
    target_json_home = xb_json_file.split('.json')[0] + '.dcm_frms'
    target_json_home = target_json_home.replace("03-annotations", "01-videos")
    os.makedirs(target_json_home, exist_ok=True)
    print(f"target_json_home={target_json_home}")
    #--02 extract json per image 
    for img_data in all_imgs_datas:
        img_objs = img_data['object']
        n = int(img_data['sopUid'].split('.')[-1])
        print(f"img({n}) has {len(img_objs)} object.")
        polygonPoints = img_objs[0]['polygonPoint']
        print(f"polygonPoint length={len(polygonPoints)}")
        filename = f"frm-{(n+1):04d}"
        points_len = int((len(polygonPoints)))

        all_points = numpy.zeros((points_len, 2))
        all_points = all_points.tolist()
        for item in polygonPoints.items():
            point_idx = int(item[0][1:])
            x0y1 = item[0][0]
            if 'x' == x0y1.lower():
                x0y1 = 0
            else:
                x0y1=1
            all_points[point_idx][x0y1] = int(item[1])
        #--03 delete zeros
        available_pts = []
        for pt in range(points_len):
            #print(f"point [{pt}]={all_points[pt]}")
            if all_points[pt][0] is None or all_points[pt][0]<10:
                #print(f"point[{pt}] is {all_points[pt]} delete it...")
                continue
            else:
                available_pts.append(all_points[pt])
        #--04 create target labelme format json
        j_target_o['imagePath']=f"{filename}.png"
        j_target_o['shapes'][0]['label']='Thyroid'
        j_target_o['shapes'][0]['points']=available_pts
        #--05 write to sidk
        target_json_name = os.path.join(target_json_home, f"{filename}.json")
        with open(target_json_name, 'w') as jfp:
            json.dump(j_target_o, jfp)
    
    

def convert_all_xiaobai_jsons(json_root_dir, count=0):
    if not os.path.exists(json_root_dir):
        print(f"Dir not exist:{json_root_dir}")
        return -1
    subdirs = []
    dirs = sorted(os.listdir(json_root_dir))
    for item in dirs:
        path = os.path.join(json_root_dir, item)
        if os.path.isdir(path):
            subdirs.append(path)
    #print(f"subdirs={subdirs}")
    
    folder_details = []
    json_f_count = 0
    for ct, json_dir in enumerate(subdirs):
        print("*"*32)
        print('Handle with {0} [{1}/]'.format(json_dir, ct + 1))
        files_in_dir = os.listdir(json_dir)
        if len(files_in_dir) < 2:
            getLogger().info(f"Empty dirctory : {json_dir}")
            continue
        else:
            for json_file in files_in_dir:
                fullfilename = os.path.join(json_dir, json_file)
                if not os.path.isfile(fullfilename):
                    continue
                #print("file in dir:", fullfilename)
                convert_to_labelme_format(fullfilename, 100)
                json_f_count = json_f_count+1
                if count != 0 and count == json_f_count:
                    print(f"\n<<<already extract {count} directory json file~")
                    return
    return 0

In [ ]:
##worked at 21/08/22
#jhomes=r"/mnt/d/workspace/03-annotations/"
#convert_all_xiaobai_jsons(jhomes, 0)

In [10]:
##working at 22/01/29
json_file=r"/mnt/d/workspace/220128-thyroid-nodules/220128-nodules/1.3.6.1.4.1.52026.66519049.5308.126.1.1.20220128001204810.json"

import json
import os
import numpy

j_target_s="""
{
  "shapes": [
    {
      "label": "\u9888\u52a8\u8109:CA",
      "line_color": null,
      "fill_color": null,
      "points": [
        [
          296,
          253
        ],
        [
          295,
          373
        ],
        [
          430,
          363
        ],
        [
          427,
          255
        ]
      ]
    }
  ],
  "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ],
  "imagePath": "frm-0001.png",
  "imageData": null
}
"""

j_target_o = json.loads(j_target_s)
j_shape_item = j_target_o["shapes"][0].copy()
j_shape_item["points"].clear()
j_shape_item["label"]=str()

#print(j_shape_item,j_shape_item)

def convert_nodule_json_v2(json_file, target_json_dir):
    #print(json_file, target_json_dir)
    #return 0
    
    if not os.path.exists(target_json_dir):
        print(f"json target directory not exist.")
        return -1
    # 01-read from nodule-json-file
    with open(json_file, 'r') as f:
        json_ = json.load(f)
        print("json has item number=", len(json_))
        for i in json_:
            print(f"\titem[{i}]")
        all_imgs_datas = json_['data']
        print("total has data number=", len(all_imgs_datas), type(all_imgs_datas))
        print(type(all_imgs_datas[0]))
        one_data_item = all_imgs_datas[0]
        print(f"\titems in \\data\\")
        for i in one_data_item:
            print(f"\t\t{i}")
#        print(f"\t\t lesions in data.")
        print("\t\t one_data_item['frameNumber']:", one_data_item['frameNumber'])
        
        
        ## extract infomation --all images.
        for one_data_item in all_imgs_datas:
            frameNumber = one_data_item['frameNumber']
            #--02 extract json per image 
            polygonPoints=[]
            filename = f"frm-{(frameNumber+1):04d}"

            #--03 create target labelme format json
            target_json=j_target_o.copy()
            target_json['imagePath']=f"{filename}.png"
            target_json['shapes'].clear()
        
            lesions = one_data_item["lesions"]
            if(len(lesions)>0):
                for lesion_idx in range(len(lesions)):
                    polygonPoints=lesions[lesion_idx]["polygonPoint"]
            
                    points_len = int((len(polygonPoints)))

                    all_points = numpy.zeros((points_len, 2))
                    all_points = polygonPoints
                    #--03.2 delete zeros
                    available_pts = []
                    for pt in range(points_len):
                        #print(f"point [{pt}]={all_points[pt]}")
                        if all_points[pt][0] is None or all_points[pt][0]<10:
                            #print(f"point[{pt}] is {all_points[pt]} delete it...")
                            continue
                        else:
                            available_pts.append(all_points[pt])
                    #--04 create target labelme format json
                    #if lesion_idx>0:
                    target_json["shapes"].append(j_shape_item.copy())
                    
                    target_json['shapes'][lesion_idx]['label']='ThyNodu'
                    target_json['shapes'][lesion_idx]['points']=available_pts
            else:
                target_json['shapes'].clear()
                    
            #--05 write to disk
            target_json_name = os.path.join(target_json_dir, f"{filename}.json")
            if os.path.exists(target_json_name):
                os.remove(target_json_name)
                print(f">>>>>delete file{target_json_name}")
            with open(target_json_name, 'w') as jfp:
                json.dump(target_json, jfp)
            
        
    return 0
  
## convert_nodule_json(json_file, ".")

In [ ]:
def convert_nodule_json_v1(json_file, target_json_dir):
    #print(json_file, target_json_dir)
    #return 0
    
    if not os.path.exists(target_json_dir):
        print(f"json target directory not exist.")
        return -1
    # 01-read from nodule-json-file
    with open(json_file, 'r') as f:
        json_ = json.load(f)
        print("json has item number=", len(json_))
        for i in json_:
            print(f"\titem[{i}]")
        all_imgs_datas = json_['data']
        print("total has data number=", len(all_imgs_datas), type(all_imgs_datas))
        print(type(all_imgs_datas[0]))
        one_data_item = all_imgs_datas[0]
        print(f"\titems in \\data\\")
        for i in one_data_item:
            print(f"\t\t{i}")
        print(f"\t\t lesions in data.")
        print("\t\t one_data_item['frameNumber']:", one_data_item['frameNumber'])
        for i in one_data_item["lesions"][0]:
            print(f"\t\t\t items in lesions:{i}, {type(i)}")
        lesions = one_data_item["lesions"]
        if(len(lesions)>0):
            polygonPoints = lesions[0]["polygonPoint"]
            print(f"polygonPoint:{polygonPoints}")
        if (len(lesions)>1):
            print("!"*20, "more than one lesion found.")
            
        #--02 extract json per image 
        for one_data_item in all_imgs_datas:
            frameNumber = one_data_item['frameNumber']
            lesions = one_data_item["lesions"]
            if(len(lesions)>0):
                polygonPoints = lesions[0]["polygonPoint"]
                #print(f"polygonPoint:{polygonPoints}")
            else:
                polygonPoints=[]
                
            if (len(lesions)>1):
                print("!"*20, "more than one lesion found.", "frameNumber=",frameNumber)
                print(f"polygonPoint length={len(polygonPoints)}")
                return -1
            filename = f"frm-{(frameNumber+1):04d}"
            points_len = int((len(polygonPoints)))

            all_points = numpy.zeros((points_len, 2))
            all_points = polygonPoints

            #--03 delete zeros
            available_pts = []
            for pt in range(points_len):
                #print(f"point [{pt}]={all_points[pt]}")
                if all_points[pt][0] is None or all_points[pt][0]<10:
                    #print(f"point[{pt}] is {all_points[pt]} delete it...")
                    continue
                else:
                    available_pts.append(all_points[pt])
            #--04 create target labelme format json
            j_target_o['imagePath']=f"{filename}.png"
            j_target_o['shapes'][0]['label']='ThyNodu'
            j_target_o['shapes'][0]['points']=available_pts
            #--05 write to disk
            target_json_name = os.path.join(target_json_dir, f"{filename}.json")
            with open(target_json_name, 'w') as jfp:
                json.dump(j_target_o, jfp)
            
        
    return 0

In [ ]:
#22/02/22 甲状腺结节的支持
import os

json_dcm_mapping=[("01","1.3.6.1.4.1.52026.66519049.5308.54.1.1.20220128180410316"),
("03","1.3.6.1.4.1.52026.66519049.5308.79.1.1.20220129115124805"),
("10","1.3.6.1.4.1.52026.66519049.5308.89.5.1.20220128232518083"),
("13","1.3.6.1.4.1.52026.66519049.5308.105.1.1.20220129145138521"),
("17","1.3.6.1.4.1.52026.66519049.5308.107.1.1.20220129191716000"),
("20","1.3.6.1.4.1.52026.66519049.5308.107.4.1.20220129014418393"),
("21","1.3.6.1.4.1.52026.66519049.5308.123.1.1.20220129235707638"),
("23","1.3.6.1.4.1.52026.66519049.5308.126.1.1.20220128001204810"),
("01","1.3.6.1.4.1.52026.66519049.5308.54.1.1.20220204151003114"),
("11","1.3.6.1.4.1.52026.66519049.5308.32.2.1.20220128210627732"),
("13","1.3.6.1.4.1.52026.66519049.5308.105.1.1.20220129145138521"),
("16","1.3.6.1.4.1.52026.66519049.5308.105.4.1.20220129020504122"),
("17","1.3.6.1.4.1.52026.66519049.5308.107.1.1.20220129191716000"),
("18","1.3.6.1.4.1.52026.66519049.5308.107.2.1.20220129220327106"),
("19","1.3.6.1.4.1.52026.66519049.5308.107.3.1.20220130002833040"),
("20","1.3.6.1.4.1.52026.66519049.5308.107.4.1.20220129014418393"),
("21","1.3.6.1.4.1.52026.66519049.5308.123.1.1.20220129235707638"),
("23","1.3.6.1.4.1.52026.66519049.5308.126.1.1.20220206234329588"),
("06","1.3.6.1.4.1.52026.66519049.5308.79.5.1.20220216000833812"),
("07","1.3.6.1.4.1.52026.66519049.5308.79.6.1.20220216150623501"),
("08","1.3.6.1.4.1.52026.66519049.5308.89.3.1.20220216161345201"),
("09","1.3.6.1.4.1.52026.66519049.5308.89.4.1.20220216164741176"),
("22","1.3.6.1.4.1.52026.66519049.5308.123.2.1.20220216172138940"),
("25","1.3.6.1.4.1.52026.100007044.2012.8.1.1.20220208225920060"),
("28","1.3.6.1.4.1.52026.66519049.5308.33.1.2.20220216232816939"),
("32","1.3.6.1.4.1.52026.66519049.5308.1.2.1.20220218142848081"),
("33","1.3.6.1.4.1.52026.66519049.5308.1.1.1.20220218161356838"),]

print("before make unique, length=", len(json_dcm_mapping))
json_dcm_mappingdict=set(json_dcm_mapping)
print("after  make unique, length=", len(json_dcm_mappingdict))

working_dir=r"/mnt/o/download/220222_thyroids"
for idx, item in enumerate(json_dcm_mappingdict):
    print(idx, item)
#     if item[0]!="17":
#         continue
    json_name=f"{item[1]}_MARK.json"
    dcm_name=f"{item[0]}.dcm"
    
    json_file_name = os.path.join(working_dir, json_name)
    dcm_file_name = os.path.join(working_dir, dcm_name)
    if not os.path.exists(json_file_name):
        print(f"json not exist:{json_file_name}")
        continue
    else:
        if not os.path.exists(dcm_file_name):
            print(f"!!!dcm not exist:{dcm_file_name}")
            continue
        else:
            failed = convert_nodule_json_v2(json_file_name,f"{dcm_file_name}_frms")
            if 0 != failed:
                print(f"\tconvert failed!!!")
            else:
                print("success,,,")
    print("*"*20, "\n")
                

        

In [12]:
#241130-process thyroid nodules from aixplorer

import os
def list_folders(directory):
    try:
        folders = []
        with os.scandir(directory) as it:
            for entry in it:
                if entry.is_dir():
                    folders.append(entry.name)
        return folders
    except Exception as e:
        print(f"Error: {e}")
        return []
    
def list_folders(directory):
    try:
        # List all entries in the directory
        entries = os.listdir(directory)
        # Filter out only the directories
        folders = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
        return folders
    except Exception as e:
        print(f"Error: {e}")
        return []

def list_File_withSuffix(directory, suffix:str="_MARK.json"):
    try:
        # List all entries in the directory
        entries = os.listdir(directory)
        # Filter out only the directories
        folders = [entry for entry in entries if entry.endswith(suffix)]
        return folders
    except Exception as e:
        print(f"Error: {e}")
        return []
    
working_dir=r"/mnt/f/240926-RayShap/241129-thyroid-datas/02-batch-10caseToLabelmeForamt"
working_dir=r"/mnt/f/240926-RayShap/241129-thyroid-datas/03-batch-72casesToLabelmeFormat"
casefolders = list_folders(working_dir)
for icase in casefolders:
    icasedir=os.path.join(working_dir, icase)
    foldersInCase=list_folders(icasedir)
    jsonInCase=list_File_withSuffix(icasedir)
    print(foldersInCase, jsonInCase)
    
    if(len(jsonInCase)<1 or len(foldersInCase)<1):
        print(f"caseDir:[{icasedir}] missing json/dicom-img-folder, ignore....")
        continue
    json_file_name = os.path.join(icasedir, jsonInCase[0])
    dcm_folder_name = os.path.join(icasedir, foldersInCase[0])
    
    failed = convert_nodule_json_v2(json_file_name, dcm_folder_name)
    if 0 != failed:
        print(f"\tconvert failed!!!")
    else:
        print("success,,,")
#os.listdir(working_dir)



['thyroidNodules_axp-013.dcm_frms'] ['1.2.250.1.204.5.667136.20211130073738532002.2.0.50.80.2.20241025191912646_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 27 <class 'list'>
<class 'dict'>
	items in \data\
		frameNumber
		lesionCount
		lesions
		 one_data_item['frameNumber']: 73
success,,,
['thyroidNodules_axp-014.dcm_frms'] ['1.2.250.1.204.5.667136.20211130081914852518.2.0.50.80.2.20241025193206142_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 64 <class 'list'>
<class 'dict'>
	items in \data\
		frameNumber
		lesionCount
		lesions
		 one_data_item['frameNumber']: 165
success,,,
['thyroidNodules_axp-015.dcm_frms'] ['1.2.250.1.204.5.667136.20211130081930159833.2.0.50.80.2.20241025222539349_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 198 <class '

success,,,
['thyroidNodules_axp-035.dcm_frms'] ['1.2.250.1.204.5.8373724313.20210224150443460916.2.0.50.80.2.20241104143020795_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 158 <class 'list'>
<class 'dict'>
	items in \data\
		frameNumber
		lesionCount
		lesions
		 one_data_item['frameNumber']: 1
success,,,
['thyroidNodules_axp-036.dcm_frms'] ['1.2.250.1.204.5.8373724313.20210315101819500879.2.0.50.80.2.20241104145004269_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 90 <class 'list'>
<class 'dict'>
	items in \data\
		frameNumber
		lesionCount
		lesions
		 one_data_item['frameNumber']: 0
success,,,
['thyroidNodules_axp-037.dcm_frms'] ['1.2.250.1.204.5.8373724313.20210220143319504010.2.0.50.80.2.20241104171057198_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data

success,,,
['thyroidNodules_axp-059.dcm_frms'] ['1.2.250.1.204.5.8373724313.20210420141419734380.2.0.50.80.2.20241108185935974_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 110 <class 'list'>
<class 'dict'>
	items in \data\
		frameNumber
		lesionCount
		lesions
		 one_data_item['frameNumber']: 0
success,,,
['thyroidNodules_axp-061.dcm_frms'] ['1.2.250.1.204.5.8373724313.20210420141730743017.2.0.50.80.2.20241108195241896_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 73 <class 'list'>
<class 'dict'>
	items in \data\
		frameNumber
		lesionCount
		lesions
		 one_data_item['frameNumber']: 9
success,,,
['thyroidNodules_axp-062.dcm_frms'] ['1.2.250.1.204.5.8373724313.20210420144402887154.2.0.50.80.2.20241108220440208_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data

success,,,
['thyroidNodules_axp-087.dcm_frms'] ['1.2.250.1.204.5.8373724313.20210416143848345628.2.0.50.80.2.20241111095056666_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 61 <class 'list'>
<class 'dict'>
	items in \data\
		frameNumber
		lesionCount
		lesions
		 one_data_item['frameNumber']: 0
success,,,
['thyroidNodules_axp-088.dcm_frms'] ['1.2.250.1.204.5.8373724313.20210416143645301067.2.0.50.80.2.20241111100402073_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data number= 47 <class 'list'>
<class 'dict'>
	items in \data\
		frameNumber
		lesionCount
		lesions
		 one_data_item['frameNumber']: 0
success,,,
['thyroidNodules_axp-089.dcm_frms'] ['1.2.250.1.204.5.667136.20211124083430227817.2.0.50.80.2.20241111171341622_MARK.json']
json has item number= 4
	item[SOPIntanceUID]
	item[data]
	item[discription]
	item[producer]
total has data numb

In [ ]:
#//220506-髋关节:HipJoint 的支持.

KeyLabel=u"髋关节:HipJoint"

import json
import os
import numpy

j_target_s=u"""
{
  "shapes": [
    {
      "label": "{KeyLabel}",
      "line_color": null,
      "fill_color": null,
      "points": [
        [
          296,
          253
        ],
        [
          295,
          373
        ],
        [
          430,
          363
        ],
        [
          427,
          255
        ]
      ]
    }
  ],
  "lineColor": [
    0,
    255,
    0,
    128
  ],
  "fillColor": [
    255,
    0,
    0,
    128
  ],
  "imagePath": "frm-0001.png",
  "imageData": null
}
"""
j_target_s=j_target_s.replace('{KeyLabel}', KeyLabel)

j_target_o = json.loads(j_target_s)
j_shape_item = j_target_o["shapes"][0].copy()
j_shape_item["points"].clear()
j_shape_item["label"]=str()

def convert_nodule_json_v3(inputJsonFile, targetJsonFormatObj, target_json_dir):
    if not os.path.exists(target_json_dir):
        print(f"json target directory not exist.")
        return -1
    # 01-read from nodule-json-file
    with open(inputJsonFile, 'r') as f:
        json_ = json.load(f)
        print("json has item number=", len(json_))
        for i in json_:
            print(f"\titem[{i}]")
        all_imgs_datas = json_['data']
        print("total has data number=", len(all_imgs_datas), type(all_imgs_datas))
        print(type(all_imgs_datas[0]))
        one_data_item = all_imgs_datas[0]
        print(f"\titems in \\data\\")
        for i in one_data_item:
            print(f"\t\t{i}")
#        print(f"\t\t lesions in data.")
        #print("\t\t one_data_item['frameNumber']:", one_data_item['frameNumber'])
        
        #count=1
        ## extract infomation --all images.
        for one_data_item in all_imgs_datas:
            frameNumber = one_data_item['frameNumber']
            #--02 extract json per image 
            polygonPoints=[]
            filename = f"frm-{(frameNumber+1):04d}"

            #--03 create target labelme format json
            target_json=j_target_o.copy()
            target_json['imagePath']=f"{filename}.png"
            target_json['shapes'].clear()
        
            lesions = one_data_item["lesions"]
            if(len(lesions)>0):
                for lesion_idx in range(len(lesions)):
                    polygonPoints=lesions[lesion_idx]["polygonPoint"]
            
                    points_len = int((len(polygonPoints)))
                    
                    all_points = numpy.zeros((points_len, 2))
                    all_points = polygonPoints
                    #--03.2 delete zeros
                    available_pts = []
                    for pt in range(points_len):
                        if all_points[pt][0] is None or all_points[pt][0]<1:
                            continue
                        else:
                            available_pts.append(all_points[pt])
                    #--04 create target labelme format json
                    #if lesion_idx>0:
                    target_json["shapes"].append(j_shape_item.copy())
                    
                    target_json['shapes'][lesion_idx]['label']=KeyLabel
                    target_json['shapes'][lesion_idx]['points']=available_pts
            else:
                target_json['shapes'].clear()
                    
            #--05 write to disk
            target_json_name = os.path.join(target_json_dir, f"{filename}.json")
            if os.path.exists(target_json_name):
                os.remove(target_json_name)
                print(f">>>>>delete file{target_json_name}")
            with open(target_json_name, 'w') as jfp:
                json.dump(target_json, jfp)
    return 0

# run it.
inputJsonFile=r"/mnt/d/workspace/220506-kuanguDemoFake/inputd/045_MARK.json"
convert_nodule_json_v3(inputJsonFile, j_shape_item, r"/mnt/d/workspace/220506-kuanguDemoFake/inputd/045.dcm_frms")